This Notebook is heavily modified from the MLCC programming project with single-digit images, here:
https://colab.research.google.com/github/google/eng-edu/blob/master/ml/cc/exercises/multi-class_classification_with_MNIST.ipynb


In [31]:
import random as rd
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
 
# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.3f}".format
 
# The following line improves formatting when ouputting NumPy arrays.
np.set_printoptions(linewidth = 200)
 
print("Loaded modules.")

Loaded modules.


In [38]:
#Load
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
np.read

In [41]:
x_test.shape

(10000, 28, 28)

In [32]:
#Load
#mnist = tf.keras.datasets.mnist
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = pd.read_csv('../input/digit-recognizer/train.csv')
x_test = pd.read_csv('../input/digit-recognizer/test.csv')

y_train = list(x_train['label'])
x_train = x_train.drop('label', axis=1)

x_train_norm = x_train/255.
x_test_norm = x_test/255.

In [33]:
# SET UP A DEEP NEURAL NET 
 
def create_DNN(learning_rate):
    """Create and compile a deep neural net."""  
    # Define the kind of model to use.
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.1)) #avoid overfitting to train set
    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=100, activation='softmax'))     
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                    loss="sparse_categorical_crossentropy",
                    metrics=['accuracy']) 
    return model 

In [34]:
 # SET UP A **CONVOLUTIONAL** NEURAL NET 
 
def create_CNN(learning_rate):
    """Create and compile a convolutional neural net."""  
    # Define the kind of model to use.
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, 6, activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.Conv2D(64, 4, activation='relu'))
    model.add(tf.keras.layers.Conv2D(128, 2, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=100, activation='softmax'))     
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                    loss="sparse_categorical_crossentropy",
                    metrics=['accuracy']) 
    return model

In [35]:
 # train on the training set with 10% held back for validation #
def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=0.1):

    history = model.fit(x=train_features, y=train_label, 
                        batch_size=batch_size,
                        epochs=epochs, shuffle=True, 
                        validation_split=validation_split)

    # Gather the model's metrics after each round of training
    epochs = history.epoch
    hist = pd.DataFrame(history.history)
    return epochs, hist

In [42]:
learning_rate = 0.001
epochs = 20
batch_size = 500
validation_split = 0.1
 
dense_NN = create_DNN(learning_rate)
 

Epoch 1/20
108/108 [==============================] - 1s 11ms/step - loss: 0.9114 - accuracy: 0.7482 - val_loss: 0.2205 - val_accuracy: 0.9363
Epoch 2/20
108/108 [==============================] - 1s 9ms/step - loss: 0.2662 - accuracy: 0.9225 - val_loss: 0.1445 - val_accuracy: 0.9610
Epoch 3/20
108/108 [==============================] - 1s 10ms/step - loss: 0.1928 - accuracy: 0.9420 - val_loss: 0.1190 - val_accuracy: 0.9662
Epoch 4/20
108/108 [==============================] - 1s 9ms/step - loss: 0.1519 - accuracy: 0.9541 - val_loss: 0.1002 - val_accuracy: 0.9712
Epoch 5/20
108/108 [==============================] - 1s 9ms/step - loss: 0.1234 - accuracy: 0.9626 - val_loss: 0.0880 - val_accuracy: 0.9737
Epoch 6/20
108/108 [==============================] - 1s 9ms/step - loss: 0.1063 - accuracy: 0.9669 - val_loss: 0.0829 - val_accuracy: 0.9755
Epoch 7/20
108/108 [==============================] - 1s 9ms/step - loss: 0.0927 - accuracy: 0.9713 - val_loss: 0.0748 - val_accuracy: 0.9770
Epoc

In [52]:
# TRAIN:
epochs_DNN, hist_DNN = train_model(dense_NN, 
                                   x_train/255., 
                                   y_train, 
                                   epochs,
                                   batch_size,
                                   validation_split)

Epoch 1/20
108/108 [==============================] - 1s 10ms/step - loss: 0.0259 - accuracy: 0.9917 - val_loss: 0.0613 - val_accuracy: 0.9817
Epoch 2/20
108/108 [==============================] - 1s 11ms/step - loss: 0.0268 - accuracy: 0.9912 - val_loss: 0.0655 - val_accuracy: 0.9810
Epoch 3/20
108/108 [==============================] - 1s 11ms/step - loss: 0.0224 - accuracy: 0.9927 - val_loss: 0.0636 - val_accuracy: 0.9837
Epoch 4/20
108/108 [==============================] - 1s 10ms/step - loss: 0.0212 - accuracy: 0.9932 - val_loss: 0.0648 - val_accuracy: 0.9845
Epoch 5/20
108/108 [==============================] - 1s 10ms/step - loss: 0.0213 - accuracy: 0.9930 - val_loss: 0.0626 - val_accuracy: 0.9842
Epoch 6/20
108/108 [==============================] - 1s 9ms/step - loss: 0.0192 - accuracy: 0.9938 - val_loss: 0.0685 - val_accuracy: 0.9833
Epoch 7/20
108/108 [==============================] - 1s 10ms/step - loss: 0.0187 - accuracy: 0.9938 - val_loss: 0.0623 - val_accuracy: 0.9832


In [49]:
# Add a channels dimension
x_train_norm = np.asarray(x_train_norm).reshape(42000, 28, 28, 1)
x_train_norm = x_train_norm.reshape(42000, 28, 28, 1)
x_test_norm = x_test_norm.reshape(28000, 28, 28, 1)

In [51]:
# Train and evalate CNN
 
convolute_NN = create_CNN(learning_rate)
 
# TRAIN:
epochs_CNN, hist_CNN = train_model(convolute_NN, x_train_norm, y_train, epochs, batch_size, validation_split)
 
#print("\n Evaluate the new model against the test set:")
#convolute_NN.evaluate(x=x_test_norm,y=y_test,batch_size=batch_size)

Epoch 1/20
13/76 [====>.........................] - ETA: 1:09 - loss: 2.8355 - accuracy: 0.1014

KeyboardInterrupt: 